<a href="https://colab.research.google.com/github/linqus/nlp-huggingface/blob/main/notebooks/unit7/nlp_7_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00


In [1]:
def any_keyword_in_string(string, keywords):
    for keyword in keywords:
        if keyword in string:
            return True
    return False

In [2]:
filters = ["pandas", "sklearn", "matplotlib", "seaborn"]
example_1 = "import numpy as np"
example_2 = "import pandas as pd"

print(
    any_keyword_in_string(example_1, filters), any_keyword_in_string(example_2, filters)
)

False True


In [5]:
from collections import defaultdict
from tqdm import tqdm
from datasets import Dataset


def filter_streaming_dataset(dataset, filters):
    filtered_dict = defaultdict(list)
    total = 0
    for sample in tqdm(iter(dataset)):
        total += 1
        if any_keyword_in_string(sample["content"], filters):
            for k, v in sample.items():
                filtered_dict[k].append(v)
    print(f"{len(filtered_dict['content'])/total:.2%} of data after filtering.")
    return Dataset.from_dict(filtered_dict)

In [6]:
# This cell will take a very long time to execute, so you should skip it and go to
# the next one!
from datasets import load_dataset

split = "train"  # "valid"
filters = ["pandas", "sklearn", "matplotlib", "seaborn"]

#data = load_dataset(f"transformersbook/codeparrot-{split}", split=split, streaming=True)
#filtered_data = filter_streaming_dataset(data, filters)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Resolving data files:   0%|          | 0/183 [00:00<?, ?it/s]

189868it [02:05, 1506.94it/s]


KeyboardInterrupt: ignored

In [8]:
from datasets import load_dataset, DatasetDict

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")


In [9]:

raw_datasets = DatasetDict(
    {
        "train": ds_train.shuffle().select(range(50000)),
        "valid": ds_valid.shuffle().select(range(500))
    }
)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 50000
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 500
    })
})

In [10]:
raw_datasets["train"][0]

{'repo_name': 'UDST/pandana',
 'path': 'pandana/network.py',
 'copies': '1',
 'size': '31374',
 'content': 'from __future__ import division, print_function\n\nimport numpy as np\nimport pandas as pd\nfrom sklearn.neighbors import KDTree\n\nfrom .cyaccess import cyaccess\nfrom .loaders import pandash5 as ph5\nimport warnings\n\n\ndef reserve_num_graphs(num):\n    """\n    This function was previously used to reserve memory space for multiple\n    graphs. It is no longer needed in Pandana 0.4+, and will be removed in a\n    future version.\n\n    Parameters\n    ----------\n    num : int\n        Number of graphs to be reserved in memory\n\n    """\n    warnings.warn(\n        "Function reserve_num_graphs() is no longer needed in Pandana 0.4+\\\n         and will be removed in a future version",\n        DeprecationWarning\n    )\n    return None\n\n\nclass Network:\n    """\n    Create the transportation network in the city.  Typical data would be\n    distance based from OpenStreetMap 

In [11]:
for key in raw_datasets["train"][0]:
    print(f"{key.upper()}: {raw_datasets['train'][0][key][:200]}")

REPO_NAME: UDST/pandana
PATH: pandana/network.py
COPIES: 1
SIZE: 31374
CONTENT: from __future__ import division, print_function

import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree

from .cyaccess import cyaccess
from .loaders import pandash5 as ph5
import
LICENSE: agpl-3.0


In [12]:
from transformers import AutoTokenizer

model_checkpoint = "huggingface-course/code-search-net-tokenizer"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


tokenizer_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/789k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/448k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [39]:
max_length = 128

inputs = tokenizer(
    raw_datasets["train"][:2]['content'],
    max_length=max_length,
    truncation=True,
    return_overflowing_tokens=True,
    return_length=True
)
print(inputs.keys())
len(inputs['input_ids'])





dict_keys(['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'])


78

In [61]:
context_length = 128
def tokenize(example):
  inputs = tokenizer(
        example['content'],
        max_length=context_length,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True
  )
  result = []
  for ids, length in zip(inputs['input_ids'], inputs['length']):
      if length == context_length:
        result.append(ids)
  return {'input_ids':result}


len( tokenize(raw_datasets["train"][1])['input_ids'] )

22

In [62]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 50000
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 500
    })
})

In [ ]:
tokenized_dataset = raw_datasets.map(tokenize, batched=True, remove_columns=raw_datasets['train'].column_names)
tokenized_dataset

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]